# ETL Project - Benjamin Aubry, Gary Fisher, and Bruce Mark

## Instructions for Importing Quandl Data Function:
#### Install Code: pip install quandl
#### Dependencies: import quandl
##### Reference: https://docs.quandl.com/docs/python-installation

## Instructions for Importing Fed Data Function:
#### Install Code: pip install datapungi_fed
#### Dependencies: import datapungi_fed as dpf
##### Reference: https://github.com/jjotterson/datapungi_fed

In [1]:
#Import Dependencies
import pandas as pd
import pymongo
import numpy as np
import requests
import sys
import os.path
import datetime as dt
import pprint

import quandl
import datapungi_fed as dpf

from quandl_apitoken import api_key
from api_keys import fred_api_key
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient

## Extract and Transform

### Quandl

In [2]:
#Quandl API call- pull copper and gold and convert to csv

try:
    copper_download = quandl.get("CHRIS/CME_HG2", authtoken=api_key)
    gold_download = quandl.get("CHRIS/CME_GC2", authtoken=api_key)
    copper_download.to_csv("data_export_files/copper_2nd.csv")
    gold_download.to_csv("data_export_files/gold_2nd.csv")

except Exception as e:
    print(e)

In [3]:
try:
    silver_download = quandl.get("CHRIS/CME_SI2", authtoken=api_key)
    crude_download = quandl.get("CHRIS/CME_CL2", authtoken=api_key)
    silver_download.to_csv("data_export_files/silver_2nd.csv")
    crude_download.to_csv("data_export_files/crude_2nd.csv")

except Exception as e:
    print(e)

In [ ]:
try:
    natgas_download = quandl.get("CHRIS/CME_NG2", authtoken=api_key)
    corn_download = quandl.get("CHRIS/CME_C2", authtoken=api_key)
    natgas_download.to_csv("data_export_files/natgas_2nd.csv")
    corn_download.to_csv("data_export_files/corn_2nd.csv")

except Exception as e:
    print(e)

In [ ]:
try:
    livecatt_download = quandl.get("CHRIS/CME_LC2", authtoken=api_key)
    wheat_download = quandl.get("CHRIS/CME_KW2", authtoken=api_key)
    livecatt_download.to_csv("data_export_files/livecatt_2nd.csv")
    wheat_download.to_csv("data_export_files/K_Wheat_2nd.csv")

except Exception as e:
    print(e)

In [ ]:
csv_file = os.path.join("data_export_files/copper_2nd.csv")
copper_raw_df = pd.read_csv(csv_file)

copper_df = copper_raw_df[["Date", "Open", "High", "Low","Settle","Volume"]].copy()
copper_df.head()

In [ ]:
csv_file = os.path.join("data_export_files/gold_2nd.csv")
gold_raw_df = pd.read_csv(csv_file)

gold_df = gold_raw_df[["Date", "Open", "High", "Low","Settle","Volume"]].copy()
gold_df.head()

In [ ]:
csv_file = os.path.join("data_export_files/silver_2nd.csv")
silver_raw_df = pd.read_csv(csv_file)

silver_df = silver_raw_df[["Date", "Open", "High", "Low","Settle","Volume"]].copy()
silver_df.head()

In [ ]:
csv_file = os.path.join("data_export_files/crude_2nd.csv")
crude_raw_df = pd.read_csv(csv_file)

crude_df = crude_raw_df[["Date", "Open", "High", "Low","Settle","Volume"]].copy()
crude_df.head()

In [ ]:
csv_file = os.path.join("data_export_files/natgas_2nd.csv")
natgas_raw_df = pd.read_csv(csv_file)

natgas_df = natgas_raw_df[["Date", "Open", "High", "Low","Settle","Volume"]].copy()
natgas_df.head()

In [ ]:
csv_file = os.path.join("data_export_files/corn_2nd.csv")
corn_raw_df = pd.read_csv(csv_file)

corn_df = corn_raw_df[["Date", "Open", "High", "Low","Settle","Volume"]].copy()
corn_df.head()

In [ ]:
csv_file = os.path.join("data_export_files/livecatt_2nd.csv")
livecatt_raw_df = pd.read_csv(csv_file)

livecatt_df = livecatt_raw_df[["Date", "Open", "High", "Low","Settle","Volume"]].copy()
livecatt_df.head()

In [ ]:
csv_file = os.path.join("data_export_files/K_Wheat_2nd.csv")
k_wheat_raw_df = pd.read_csv(csv_file)

k_wheat_df = k_wheat_raw_df[["Date", "Open", "High", "Low","Settle","Volume"]].copy()
k_wheat_df.head()

### St. Louis Federal Reserve

In [ ]:
data = dpf.data(fred_api_key)

In [ ]:
keys = ["DBAA", "BAMLH0A3HYCEY", "USD3MTD156N", "DGS10", "T10Y2Y", "gdp", "gnp"]


df0 = data.series("AAA")

for key in keys:
    df1 = data.series(key)
    df0 = pd.merge(df0, df1, on="date", how="outer")
      
df0 = df0.reset_index()
df0.head()

In [ ]:
master_df = pd.DataFrame({
    "Date": df0["date"],
    "AAA Corp Yield": df0["AAA"],
    "BBB Corp Yield": df0["DBAA"],
    "CCC Corp High Yield": df0["BAMLH0A3HYCEY"],
    "3-Month LIBOR": df0["USD3MTD156N"],
    "10-Yr Treas Yld": df0["DGS10"],
    "10Yr - 2Yr Treas Yld": df0["T10Y2Y"],
    "US GDP": df0["gdp"],
    "US GNP": df0["gnp"]
})

master_df = master_df.sort_values(by=["Date"])
master_df.head(5)

In [ ]:
column_value = master_df[master_df["Date"] < "2019-04-01"].index
master_df.drop(column_value, inplace=True)
master_df.set_index("Date", inplace=True)
master_df.head()

### Forexfactory

In [ ]:
url = 'https://www.forexfactory.com/calendar?month=last'

In [ ]:
response = requests.get(url)

In [ ]:
data = response.text
soup = bs(data, 'lxml')

In [ ]:
# Searching for elements in the table
table = soup.find('table', class_='calendar__table')
#print(table.prettify())

In [ ]:
# Looping through the calendar table
list_of_rows = []


for row in table.find_all('tr', {'data-eventid':True}):
    list_of_cells = []
    
    #Filtering high-impact events
    for cell in row.find_all('td', class_=[
          'calendar__cell calendar__date date',
          'calendar__cell calendar__currency currency', 
          'calendar__cell calendar__event event', 
          'calendar__cell calendar__actual actual', 
          'calendar__cell calendar__forecast forecast', 
          'calendar__cell calendar__previous previous']):
            
        list_of_cells.append(cell.text)
    list_of_rows.append(list_of_cells)


In [ ]:
df_calendar = pd.DataFrame(list_of_rows, columns=['Date','Country','Event','Actual','Forecast','Previous'])
df_calendar.iloc[:,1] = df_calendar.iloc[:,1].str.split('\n').str[1]

df_calendar = df_calendar.reset_index()
df_calendar = df_calendar.set_index(df_calendar.columns[1])
df_calendar = df_calendar.drop(columns=['index'])
df_calendar

In [ ]:
df_calendar

In [ ]:
new_calendar = df_calendar

In [ ]:
new_calendar = new_calendar.dropna(how='all')
new_calendar

In [ ]:
# Resetting the index
new_calendar = new_calendar.reset_index()

In [ ]:
# Applying the function to get rid of the day and convert the string to date
new_calendar['Date'] = [date[4:] for date in new_calendar['Date'].to_list()]

In [ ]:
# Check the uniqueness of the dates
new_calendar['Date'].unique()

In [ ]:
new_calendar

In [ ]:
# Replacing blank cells with NaN
new_calendar['Date'] = new_calendar['Date'].replace( '',np.nan).fillna(method='ffill')

In [ ]:
new_calendar['Date'] = new_calendar['Date'] + '20'

In [ ]:
new_calendar['Date']=  pd.to_datetime(new_calendar['Date'], format='%b %d %y')

In [ ]:
new_calendar

In [ ]:
new_calendar_group = new_calendar.groupby(["Date", "Country", "Event"])
#new_calendar_group.first().head(50)

In [ ]:
# Setting up the index
#new_calendar.set_index('Date')

## More Transforming and Load to MongoDB

### Create MongoDB

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'Economics' database in Mongo
db = client.economics_db

In [ ]:
# Declare the collections
collection_commod_copper = db.commodities_copper
collection_commod_gold = db.commodities_gold
collection_commod_silver = db.commodities_silver
collection_commod_crude = db.commodities_crude
collection_commod_natgas = db.commodities_natgas
collection_commod_corn = db.commodities_corn
collection_commod_livecatt = db.commodities_livecatt
collection_commod_wheat = db.commodities_wheat
collection_fed = db.fed_data
collection_events = db.event_data

### Quandl

In [ ]:
#setindex on other df's 
copper_df = copper_df.set_index("Date")
gold_df = gold_df.set_index("Date")
silver_df = silver_df.set_index("Date")
crude_df = crude_df.set_index("Date")
natgas_df = natgas_df.set_index("Date")
corn_df = corn_df.set_index("Date")
livecatt_df = livecatt_df.set_index("Date")
k_wheat_df = k_wheat_df.set_index("Date")

In [ ]:
#create additional dictionaries for mongo load
#index in () is how you order this
copper_dict = copper_df.to_dict("index")
gold_dict = gold_df.to_dict("index")
silver_dict = silver_df.to_dict("index")
crude_dict = crude_df.to_dict("index")
natgas_dict = natgas_df.to_dict("index")
corn_dict = corn_df.to_dict("index")
livecatt_dict = livecatt_df.to_dict("index")
k_wheat_dict = k_wheat_df.to_dict("index")

In [ ]:
#load dict to mongo
collection_commod_copper.insert_many([{str(k):b} for k, b in copper_dict.items()])

In [ ]:
#load dict to mongo
collection_commod_gold.insert_many([{str(k):b} for k, b in gold_dict.items()])

In [ ]:
#load dict to mongo
collection_commod_silver.insert_many([{str(k):b} for k, b in silver_dict.items()])

In [ ]:
#load dict to mongo
collection_commod_crude.insert_many([{str(k):b} for k, b in crude_dict.items()])

In [ ]:
#load dict to mongo
collection_commod_natgas.insert_many([{str(k):b} for k, b in natgas_dict.items()])

In [ ]:
#load dict to mongo
collection_commod_corn.insert_many([{str(k):b} for k, b in corn_dict.items()])

In [ ]:
#load dict to mongo
collection_commod_livecatt.insert_many([{str(k):b} for k, b in livecatt_dict.items()])

In [ ]:
#load dict to mongo
collection_commod_wheat.insert_many([{str(k):b} for k, b in k_wheat_dict.items()])

### St. Louis Federal Reserve

In [ ]:
master_df_dict = master_df.to_dict("index")

In [ ]:
first = dict(list(master_df_dict.items())[0:2])
first

In [ ]:
collection_fed.insert_many([{str(k):b} for k, b in master_df_dict.items()])

In [ ]:
for doc in collection_fed.find()[0:3]:
    print(doc)

### Forexfactory

In [ ]:
# Converting the dataframe to a calendar 
event_data_dict = new_calendar.to_dict("index")

In [ ]:
second = dict(list(event_data_dict.items())[0:2])
second

In [ ]:
collection_events.insert_many([{str(k):b} for k, b in event_data_dict.items()])

In [ ]:
for doc in collection_events.find()[0:3]:
    print(doc)